In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import time
import pickle
# from imblearn.over_sampling import SMOTE
# from tensorflow.keras.regularizers import  l1,l2
from collections import Counter
import tensorflow as tf

In [ ]:
feature_vector_embedding=20
voc_size=30000
sentence_length=20

In [ ]:
class Client:
    feature_vector_embedding=20
    voc_size=30000
    sentence_length=20

    
    def __init__(self, dataset_x, dataset_y, epoch_number, learning_rate,weights,batch,x_valid,y_valid):
        self.dataset_x=dataset_x
        self.dataset_y=dataset_y
        self.epoch_number=epoch_number
        self.learning_rate=learning_rate
        self.weights=weights
        self.batch=batch
        self.x_valid=x_valid
        self.y_valid=y_valid
        
        
    def train(self): 
        import numpy as np
        # import pandas as pd
        import matplotlib as plt
        from tensorflow import keras
        
        model = keras.models.Sequential([
        keras.layers.Embedding(voc_size,feature_vector_embedding, input_length=sentence_length),
        keras.layers.GRU(30, return_sequences=True),
        keras.layers.Dropout(0.5),
        keras.layers.GRU(20, return_sequences=True),
        keras.layers.GRU(10, return_sequences=True),
        keras.layers.GRU(5),
        keras.layers.Dense(3, activation="softmax")
    ])
        
        #setting weight of the model
        model.set_weights(self.weights)
        
        model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
        history=model.fit(self.dataset_x, self.dataset_y, validation_data=(self.x_valid, self.y_valid), epochs=self.epoch_number, batch_size=self.batch)        
       
        #getting the final_weight
        output_weight=model.get_weights()        
        return output_weight

# Redo

In [ ]:
file = open("/content/drive/MyDrive/Datasets/x_train",'rb')
x_train= pickle.load(file)
x_train=x_train[:75000,:]

file = open("/content/drive/MyDrive/Datasets/y_train",'rb')
y_train= pickle.load(file)
y_train=y_train[:75000]

file = open("/content/drive/MyDrive/Datasets/x_test",'rb')
x_test= pickle.load(file)
x_test=x_test[:25000,:]

file = open("/content/drive/MyDrive/Datasets/y_test",'rb')
y_test= pickle.load(file)
y_test=y_test[:25000]



In [ ]:
print(type(x_train))
print(type(y_train))
print(x_train.shape)
print(x_test.shape)
print(y_test)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(75000, 20)
(25000, 20)
[0 2 1 ... 0 0 2]


In [ ]:
def divide_without_label(parts, X_train_full,y_train_full):
    
        each_part_number=int(len(X_train_full)/parts)
        list_x_train=[]
        list_y_train=[]
        
        number_list=[]
        number_list.append(0)
        for x in range(1, parts+1):
            number_list.append(each_part_number*x)
        
        
        for x in range(0,parts):
            data_x=X_train_full[number_list[x]:number_list[x+1]]
            data_y=y_train_full[number_list[x]:number_list[x+1]]
            list_x_train.append(data_x)
            list_y_train.append(data_y)
            
        return list_x_train, list_y_train

In [ ]:
x_data,y_data=divide_without_label(7, x_train, y_train)

In [ ]:
#function for averaging
def get_model():
  model = keras.models.Sequential([
          keras.layers.Embedding(voc_size,feature_vector_embedding, input_length=sentence_length),
          keras.layers.GRU(30, return_sequences=True),
          keras.layers.Dropout(0.5),
          keras.layers.GRU(20, return_sequences=True),
          keras.layers.GRU(10, return_sequences=True),
          keras.layers.GRU(5),
          keras.layers.Dense(3, activation="softmax")
      ])
    
  return model

In [ ]:
def model_average(client_weights):
    average_weight_list=[]
    for index1 in range(len(client_weights[0])):
        layer_weights=[]
        for index2 in range(len(client_weights)):
            weights=client_weights[index2][index1]
            layer_weights.append(weights)
        average_weight=np.mean(np.array([x for x in layer_weights]), axis=0)
        average_weight_list.append(average_weight)
    return average_weight_list

In [ ]:
def create_model():
    model=get_model()
    weight=model.get_weights()
    return weight

In [ ]:
def evaluate_model(accuracy_list,weight,learning_rate):
    model=get_model()  
    model.set_weights(weight)
    model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
    result=model.evaluate(x_test, y_test)
            
    if len(accuracy_list)==0:
        accuracy_list.append(0)
        if result[1] > accuracy_list[len(accuracy_list)-1]:
            return True,result[1]
        
    elif result[1] > accuracy_list[len(accuracy_list)-1]:
            return True,result[1]
    else:
        return False,result[1]

In [ ]:
accuracy_list_global=[]
def train_server(training_rounds,epoch,batch,learning_rate):
    
    accuracy_list=[]
    client_weight_for_sending=[]
    
    for index1 in range(1,training_rounds):
        print('Training for round ', index1, 'started')
        client_weights_tobe_averaged=[]
        for index in range(len(y_data)):
            print('-------Client-------', index)
            if index1==1:
                print('Sharing Initial Global Model with Random Weight Initialization')
                initial_weight=create_model()
                client=Client(x_data[index],y_data[index],epoch,learning_rate,initial_weight,batch,x_test, y_test)
                weight=client.train()
                client_weights_tobe_averaged.append(weight)
            else:
                client=Client(x_data[index],y_data[index],epoch,learning_rate,client_weight_for_sending[index1-2],batch,x_test, y_test)
                weight=client.train()
                client_weights_tobe_averaged.append(weight)
    
        #calculating the avearge weight from all the clients
        client_average_weight=model_average(client_weights_tobe_averaged)
        client_weight_for_sending.append(client_average_weight)


        #validating the model with avearge weight
        model=get_model()

        model.set_weights(client_average_weight)
        model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.SGD(lr=learning_rate),metrics=['accuracy'])
        result=model.evaluate(x_test, y_test)
        accuracy=result[1]
        print('#######-----Acccuracy for round ', index1, 'is ', accuracy, ' ------########')
        accuracy_list.append(accuracy)
        accuracy_list_global.append(accuracy)
        
    return accuracy_list, client_weight_for_sending

In [ ]:
def train_main():
    start=time.time()
    training_accuracy,weights=train_server(10,5,32,0.01)
    end=time.time()
    print('TOTAL TIME = ', end-start)

In [ ]:
if __name__=="__main__":
    train_main()